In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [15]:
GROUP_QUOTA = int((1070 / 6)) # 6 groups, 1070 students
GROUPS = ["1", "3", "4", "5", "6", "7"]

print(f"Quota per group: {GROUP_QUOTA}")
print(f"Groups: {GROUPS}")

Quota per group: 178
Groups: ['1', '3', '4', '5', '6', '7']


In [16]:
df = pd.read_csv("../data/team_processed.csv")

In [17]:
df

,id,group_pref,member_count
0,ec3766c1-b3ee-4873-9048-b7f7fa17ceba,"6,1,3,5,7,4",2
1,738b973c-4cbf-4125-b50e-bcc98f89f77a,"6,1,3,7,4,5",2
2,a394c8ef-98d7-49b5-beb3-652b08cd6482,"4,3,6,7,1,5",1
3,3620b751-1dcc-4305-8871-b897947ba300,"3,4,7,1,6,5",1
4,5602bd74-811a-4e99-9708-179254d9f090,"3,6,7,1,4,5",2
...,...,...,...
648,959f881a-310f-4e23-b715-1cef16e97869,"3,4,1,5,7,6",1
649,b7edde69-726b-4e09-a6c3-9d9b95fd9b4a,"6,5,7,1,3,4",1
650,77b3a72b-fd79-4d5b-9537-2ad7df143cbd,"4,6,3,1,7,5",1
651,1abaf1d3-b2df-46eb-8ed2-652e98151862,"4,1,6,3,7,5",1


In [18]:
# split col group_pref in to multiple columns where each column is a group order and value is the group number
df_group_pref = df["group_pref"].str.split(",", expand=True)
df_group_pref.columns = [f"group_order_{i+1}" for i in range(df_group_pref.shape[1])]
df = df.join(df_group_pref)
# add blank columns result
df["result"] = None
df

,id,group_pref,member_count,group_order_1,group_order_2,group_order_3,group_order_4,group_order_5,group_order_6,result
0,ec3766c1-b3ee-4873-9048-b7f7fa17ceba,"6,1,3,5,7,4",2,6,1,3,5,7,4,None
1,738b973c-4cbf-4125-b50e-bcc98f89f77a,"6,1,3,7,4,5",2,6,1,3,7,4,5,None
2,a394c8ef-98d7-49b5-beb3-652b08cd6482,"4,3,6,7,1,5",1,4,3,6,7,1,5,None
3,3620b751-1dcc-4305-8871-b897947ba300,"3,4,7,1,6,5",1,3,4,7,1,6,5,None
4,5602bd74-811a-4e99-9708-179254d9f090,"3,6,7,1,4,5",2,3,6,7,1,4,5,None
...,...,...,...,...,...,...,...,...,...,...
648,959f881a-310f-4e23-b715-1cef16e97869,"3,4,1,5,7,6",1,3,4,1,5,7,6,None
649,b7edde69-726b-4e09-a6c3-9d9b95fd9b4a,"6,5,7,1,3,4",1,6,5,7,1,3,4,None
650,77b3a72b-fd79-4d5b-9537-2ad7df143cbd,"4,6,3,1,7,5",1,4,6,3,1,7,5,None
651,1abaf1d3-b2df-46eb-8ed2-652e98151862,"4,1,6,3,7,5",1,4,1,6,3,7,5,None


## รอบที่ 1

In [19]:
for group in GROUPS:
    print(f"Group {group} has {df[df['group_order_1'] == group].shape[0]} teams in first order, total students: {df[df['group_order_1'] == group]['member_count'].sum()}")

Group 1 has 43 teams in first order, total students: 74
Group 3 has 131 teams in first order, total students: 216
Group 4 has 143 teams in first order, total students: 226
Group 5 has 41 teams in first order, total students: 55
Group 6 has 196 teams in first order, total students: 333
Group 7 has 99 teams in first order, total students: 166


In [20]:
# สำหรับแต่ละกรุ๊ป ทีมไหนที่มีคนเลือกอันดับ 1 แล้วไม่เกินโควต้า ก็ให้ไปเลย
for group in GROUPS:
  student_preferred_sum = df[df["group_order_1"] == group]["member_count"].sum()
  if student_preferred_sum <= GROUP_QUOTA:
    df.loc[df["group_order_1"] == group, "result"] = group
    print(f"Group {group} has {student_preferred_sum} students, assigned all to group {group}")
  else:
    print(f"Group {group} has {student_preferred_sum} students, not enough quota, randomly selecting teams to assign to group {group}")
    current_students_count = df[df["result"] == group]["member_count"].sum()
    df_teams_with_group_first_order = df[df["group_order_1"] == group]
    # shffle row order
    df_teams_with_group_first_order = df_teams_with_group_first_order.sample(frac=1)
    for index, row in df_teams_with_group_first_order.iterrows():
      if current_students_count <= GROUP_QUOTA:
        df.at[index, "result"] = group
        current_students_count += row["member_count"]
        print(
            f"Assigning team {index}:{row['id']} ({row['member_count']} pp) to group {group}, current count: {current_students_count}, quota left: {GROUP_QUOTA - current_students_count}")
      else:
        print(f"Group {group} has reached quota, stop assigning")
        break
      
# Check if there are any students left unassigned
unassigned_teams = df[df["result"].isnull()]
if not unassigned_teams.empty:
    print(f"There are {unassigned_teams.shape[0]} teams left unassigned.")
else:
    print("All teams have been assigned to groups.")
  
for group in GROUPS:
    print(f"Group {group} has {df[df['result'] == group].shape[0]} teams assigned, total students: {df[df['result'] == group]['member_count'].sum()}")  
df

Group 1 has 74 students, assigned all to group 1
Group 3 has 216 students, not enough quota, randomly selecting teams to assign to group 3
Assigning team 124:278ba432-a8a4-4256-9d78-514132f4012e (2 pp) to group 3, current count: 2, quota left: 176
Assigning team 625:08c7ace3-4eac-4e00-934e-6cd56b93cc9b (1 pp) to group 3, current count: 3, quota left: 175
Assigning team 190:cfe77fc9-879f-4d4b-a498-0a81cad4cf9a (3 pp) to group 3, current count: 6, quota left: 172
Assigning team 483:99d7a98d-9258-4e06-a0dc-45ea8a3304a3 (1 pp) to group 3, current count: 7, quota left: 171
Assigning team 323:77a63c9e-904b-42f0-91a2-a1b6e8be13d7 (1 pp) to group 3, current count: 8, quota left: 170
Assigning team 54:bc506bca-ebc2-4367-9719-3bf6439a56b8 (3 pp) to group 3, current count: 11, quota left: 167
Assigning team 241:60b5a06a-d98a-49d7-bf5a-a85ef31fdef5 (1 pp) to group 3, current count: 12, quota left: 166
Assigning team 378:6aa15d87-9ec6-4426-8676-0757f1aa05fb (3 pp) to group 3, current count: 15, quo

,id,group_pref,member_count,group_order_1,group_order_2,group_order_3,group_order_4,group_order_5,group_order_6,result
0,ec3766c1-b3ee-4873-9048-b7f7fa17ceba,"6,1,3,5,7,4",2,6,1,3,5,7,4,6
1,738b973c-4cbf-4125-b50e-bcc98f89f77a,"6,1,3,7,4,5",2,6,1,3,7,4,5,6
2,a394c8ef-98d7-49b5-beb3-652b08cd6482,"4,3,6,7,1,5",1,4,3,6,7,1,5,4
3,3620b751-1dcc-4305-8871-b897947ba300,"3,4,7,1,6,5",1,3,4,7,1,6,5,3
4,5602bd74-811a-4e99-9708-179254d9f090,"3,6,7,1,4,5",2,3,6,7,1,4,5,3
...,...,...,...,...,...,...,...,...,...,...
648,959f881a-310f-4e23-b715-1cef16e97869,"3,4,1,5,7,6",1,3,4,1,5,7,6,3
649,b7edde69-726b-4e09-a6c3-9d9b95fd9b4a,"6,5,7,1,3,4",1,6,5,7,1,3,4,None
650,77b3a72b-fd79-4d5b-9537-2ad7df143cbd,"4,6,3,1,7,5",1,4,6,3,1,7,5,4
651,1abaf1d3-b2df-46eb-8ed2-652e98151862,"4,1,6,3,7,5",1,4,1,6,3,7,5,4


# รอบ 2

In [21]:
# เราจัดการการเลือกอันดับ 1 เสร็จแล้ว ทีนี้จัดคนไม่มีกลุ่มไปที่กลุ่มอันดับ 2 ที่เลือกไว้

for group in GROUPS:
    print("="*80)
    # check if group is already filled to the quota already
    if df[df["result"] == group]["member_count"].sum() > GROUP_QUOTA:
        print(f"Group {group} is already filled to the quota, skipping")
        continue
    print(f"Processing group {group} for second order assignment")
    # get teams that have group_order_2 as group and not assigned to any group yet
    df_teams_with_group_second_order = df[(df["group_order_2"] == group) & (df["result"].isnull())]
    # shuffle row order
    df_teams_with_group_second_order = df_teams_with_group_second_order.sample(frac=1)
    current_students_count = df[df["result"] == group]["member_count"].sum()
    for index, row in df_teams_with_group_second_order.iterrows():
        if current_students_count < GROUP_QUOTA:
            df.at[index, "result"] = group
            current_students_count += row["member_count"]
            print(
                f"Assigning team {index}:{row['id']} ({row['member_count']} pp) to group {group}, current count: {current_students_count}, quota left: {GROUP_QUOTA - current_students_count}")
        else:
            print(f"Group {group} has reached quota, stop assigning")
            break

unassigned_teams = df[df["result"].isnull()]
if not unassigned_teams.empty:
    print(f"There are {unassigned_teams.shape[0]} teams left unassigned.")
else:
    print("All teams have been assigned to groups.")
    
df

Processing group 1 for second order assignment
Assigning team 436:37e71c5b-ba0b-4f02-ae2b-16bed9e19482 (1 pp) to group 1, current count: 75, quota left: 103
Assigning team 291:bf6b5792-18a2-478d-94c6-a7914a3b2ccf (1 pp) to group 1, current count: 76, quota left: 102
Assigning team 560:d24c3607-2891-46d4-8c48-776a7b049798 (1 pp) to group 1, current count: 77, quota left: 101
Assigning team 262:f8696c9f-f125-40d4-81c3-32aa2eb9a8aa (1 pp) to group 1, current count: 78, quota left: 100
Assigning team 314:cddd66fd-d36e-49cf-a6a0-6d21d5eaacf6 (1 pp) to group 1, current count: 79, quota left: 99
Assigning team 56:f8d0c839-7ba2-437c-a3fe-8b3584f2e114 (1 pp) to group 1, current count: 80, quota left: 98
Assigning team 145:ae2be29f-fb2d-4772-9fe7-2c6c66e2703b (3 pp) to group 1, current count: 83, quota left: 95
Assigning team 201:560f9771-4a92-496d-ba38-184867680c46 (3 pp) to group 1, current count: 86, quota left: 92
Assigning team 497:b2e7c371-fd5d-4547-85d6-9f0ebf5e0b12 (1 pp) to group 1, cur

,id,group_pref,member_count,group_order_1,group_order_2,group_order_3,group_order_4,group_order_5,group_order_6,result
0,ec3766c1-b3ee-4873-9048-b7f7fa17ceba,"6,1,3,5,7,4",2,6,1,3,5,7,4,6
1,738b973c-4cbf-4125-b50e-bcc98f89f77a,"6,1,3,7,4,5",2,6,1,3,7,4,5,6
2,a394c8ef-98d7-49b5-beb3-652b08cd6482,"4,3,6,7,1,5",1,4,3,6,7,1,5,4
3,3620b751-1dcc-4305-8871-b897947ba300,"3,4,7,1,6,5",1,3,4,7,1,6,5,3
4,5602bd74-811a-4e99-9708-179254d9f090,"3,6,7,1,4,5",2,3,6,7,1,4,5,3
...,...,...,...,...,...,...,...,...,...,...
648,959f881a-310f-4e23-b715-1cef16e97869,"3,4,1,5,7,6",1,3,4,1,5,7,6,3
649,b7edde69-726b-4e09-a6c3-9d9b95fd9b4a,"6,5,7,1,3,4",1,6,5,7,1,3,4,5
650,77b3a72b-fd79-4d5b-9537-2ad7df143cbd,"4,6,3,1,7,5",1,4,6,3,1,7,5,4
651,1abaf1d3-b2df-46eb-8ed2-652e98151862,"4,1,6,3,7,5",1,4,1,6,3,7,5,4


# รอบ 3

In [22]:
for group in GROUPS:
    print("="*80)
    # check if group is already filled to the quota already
    if df[df["result"] == group]["member_count"].sum() > GROUP_QUOTA:
        print(f"Group {group} is already filled to the quota, skipping")
        continue
    print(f"Processing group {group} for third order assignment")
    # get teams that have group_order_3 as group and not assigned to any group yet
    df_teams_with_group_third_order = df[(
        df["group_order_3"] == group) & (df["result"].isnull())]
    # shuffle row order
    df_teams_with_group_third_order = df_teams_with_group_third_order.sample(
        frac=1)
    current_students_count = df[df["result"] == group]["member_count"].sum()
    for index, row in df_teams_with_group_third_order.iterrows():
        if current_students_count < GROUP_QUOTA:
            df.at[index, "result"] = group
            current_students_count += row["member_count"]
            print(
                f"Assigning team {index}:{row['id']} ({row['member_count']} pp) to group {group}, current count: {current_students_count}, quota left: {GROUP_QUOTA - current_students_count}")
        else:
            print(f"Group {group} has reached quota, stop assigning")
            break

unassigned_teams = df[df["result"].isnull()]
if not unassigned_teams.empty:
    print(f"There are {unassigned_teams.shape[0]} teams left unassigned.")
else:
    print("All teams have been assigned to groups.")

df

Processing group 1 for third order assignment
Assigning team 338:0b69fba6-a1e2-41df-aeab-2b0db2b0e3f2 (1 pp) to group 1, current count: 92, quota left: 86
Assigning team 282:8eb1f3f1-d6b2-4423-84b3-affd6805a3c2 (2 pp) to group 1, current count: 94, quota left: 84
Assigning team 639:60548c20-2fe5-4d89-aee4-5fbc12802bd5 (1 pp) to group 1, current count: 95, quota left: 83
Assigning team 39:f5416098-74f7-4b81-86d5-2c584da2a44b (2 pp) to group 1, current count: 97, quota left: 81
Assigning team 11:d6381cb3-8a50-4916-a9fe-6d03c31c11c4 (2 pp) to group 1, current count: 99, quota left: 79
Assigning team 161:cb9bbb86-5180-4b05-8e46-81b425c65cdd (2 pp) to group 1, current count: 101, quota left: 77
Assigning team 199:71a20e2a-5751-4c9d-a465-8a05a8f38200 (1 pp) to group 1, current count: 102, quota left: 76
Assigning team 257:cdfebcd0-fa5c-42bb-abb8-3dca7e93601b (3 pp) to group 1, current count: 105, quota left: 73
Assigning team 368:fdd80e55-7d7a-4298-9427-8bc25b7bfec3 (1 pp) to group 1, curren

,id,group_pref,member_count,group_order_1,group_order_2,group_order_3,group_order_4,group_order_5,group_order_6,result
0,ec3766c1-b3ee-4873-9048-b7f7fa17ceba,"6,1,3,5,7,4",2,6,1,3,5,7,4,6
1,738b973c-4cbf-4125-b50e-bcc98f89f77a,"6,1,3,7,4,5",2,6,1,3,7,4,5,6
2,a394c8ef-98d7-49b5-beb3-652b08cd6482,"4,3,6,7,1,5",1,4,3,6,7,1,5,4
3,3620b751-1dcc-4305-8871-b897947ba300,"3,4,7,1,6,5",1,3,4,7,1,6,5,3
4,5602bd74-811a-4e99-9708-179254d9f090,"3,6,7,1,4,5",2,3,6,7,1,4,5,3
...,...,...,...,...,...,...,...,...,...,...
648,959f881a-310f-4e23-b715-1cef16e97869,"3,4,1,5,7,6",1,3,4,1,5,7,6,3
649,b7edde69-726b-4e09-a6c3-9d9b95fd9b4a,"6,5,7,1,3,4",1,6,5,7,1,3,4,5
650,77b3a72b-fd79-4d5b-9537-2ad7df143cbd,"4,6,3,1,7,5",1,4,6,3,1,7,5,4
651,1abaf1d3-b2df-46eb-8ed2-652e98151862,"4,1,6,3,7,5",1,4,1,6,3,7,5,4


# รอบ 4

In [23]:
for group in GROUPS:
    print("="*80)
    # check if group is already filled to the quota already
    if df[df["result"] == group]["member_count"].sum() > GROUP_QUOTA:
        print(f"Group {group} is already filled to the quota, skipping")
        continue
    print(f"Processing group {group} for fourth order assignment")
    # get teams that have group_order_4 as group and not assigned to any group yet
    df_teams_with_group_fourth_order = df[(
        df["group_order_4"] == group) & (df["result"].isnull())]
    # shuffle row order
    df_teams_with_group_fourth_order = df_teams_with_group_fourth_order.sample(
        frac=1)
    current_students_count = df[df["result"] == group]["member_count"].sum()
    for index, row in df_teams_with_group_fourth_order.iterrows():
        if current_students_count < GROUP_QUOTA:
            df.at[index, "result"] = group
            current_students_count += row["member_count"]
            print(
                f"Assigning team {index}:{row['id']} ({row['member_count']} pp) to group {group}, current count: {current_students_count}, quota left: {GROUP_QUOTA - current_students_count}")
        else:
            print(f"Group {group} has reached quota, stop assigning")
            break

unassigned_teams = df[df["result"].isnull()]
if not unassigned_teams.empty:
    print(f"There are {unassigned_teams.shape[0]} teams left unassigned.")
else:
    print("All teams have been assigned to groups.")

df

Processing group 1 for fourth order assignment
Assigning team 317:eae3f053-bc30-4308-90b1-24b0987fc6e3 (3 pp) to group 1, current count: 138, quota left: 40
Assigning team 208:fc744575-6b76-4c0f-82e8-4a55f54358c3 (1 pp) to group 1, current count: 139, quota left: 39
Assigning team 466:92b6a231-f5eb-49eb-91ee-e491257c997c (3 pp) to group 1, current count: 142, quota left: 36
Assigning team 498:2a9cafab-1e50-4f48-9da3-fde0db5d9434 (1 pp) to group 1, current count: 143, quota left: 35
Assigning team 131:849d99c3-b6f7-41a8-9102-bf6c6e8e5eed (2 pp) to group 1, current count: 145, quota left: 33
Assigning team 511:e00b10f5-3f60-48de-ac18-5b387137b8a8 (2 pp) to group 1, current count: 147, quota left: 31
Assigning team 26:186b25da-ff0d-4dde-b534-357070d412ea (3 pp) to group 1, current count: 150, quota left: 28
Assigning team 523:06373a73-a085-4da4-b6e0-7e3b205413ed (1 pp) to group 1, current count: 151, quota left: 27
Assigning team 118:7d0595b3-9ccf-49ce-9ca7-62a261f59f22 (1 pp) to group 1,

,id,group_pref,member_count,group_order_1,group_order_2,group_order_3,group_order_4,group_order_5,group_order_6,result
0,ec3766c1-b3ee-4873-9048-b7f7fa17ceba,"6,1,3,5,7,4",2,6,1,3,5,7,4,6
1,738b973c-4cbf-4125-b50e-bcc98f89f77a,"6,1,3,7,4,5",2,6,1,3,7,4,5,6
2,a394c8ef-98d7-49b5-beb3-652b08cd6482,"4,3,6,7,1,5",1,4,3,6,7,1,5,4
3,3620b751-1dcc-4305-8871-b897947ba300,"3,4,7,1,6,5",1,3,4,7,1,6,5,3
4,5602bd74-811a-4e99-9708-179254d9f090,"3,6,7,1,4,5",2,3,6,7,1,4,5,3
...,...,...,...,...,...,...,...,...,...,...
648,959f881a-310f-4e23-b715-1cef16e97869,"3,4,1,5,7,6",1,3,4,1,5,7,6,3
649,b7edde69-726b-4e09-a6c3-9d9b95fd9b4a,"6,5,7,1,3,4",1,6,5,7,1,3,4,5
650,77b3a72b-fd79-4d5b-9537-2ad7df143cbd,"4,6,3,1,7,5",1,4,6,3,1,7,5,4
651,1abaf1d3-b2df-46eb-8ed2-652e98151862,"4,1,6,3,7,5",1,4,1,6,3,7,5,4


# รอบ 5

In [24]:
for group in GROUPS:
    print("="*80)
    # check if group is already filled to the quota already
    if df[df["result"] == group]["member_count"].sum() > GROUP_QUOTA:
        print(f"Group {group} is already filled to the quota, skipping")
        continue
    print(f"Processing group {group} for fifth order assignment")
    # get teams that have group_order_5 as group and not assigned to any group yet
    df_teams_with_group_fifth_order = df[(
        df["group_order_5"] == group) & (df["result"].isnull())]
    # shuffle row order
    df_teams_with_group_fifth_order = df_teams_with_group_fifth_order.sample(
        frac=1)
    current_students_count = df[df["result"] == group]["member_count"].sum()
    for index, row in df_teams_with_group_fifth_order.iterrows():
        if current_students_count < GROUP_QUOTA:
            df.at[index, "result"] = group
            current_students_count += row["member_count"]
            print(
                f"Assigning team {index}:{row['id']} ({row['member_count']} pp) to group {group}, current count: {current_students_count}, quota left: {GROUP_QUOTA - current_students_count}")
        else:
            print(f"Group {group} has reached quota, stop assigning")
            break

unassigned_teams = df[df["result"].isnull()]
if not unassigned_teams.empty:
    print(f"There are {unassigned_teams.shape[0]} teams left unassigned.")
else:
    print("All teams have been assigned to groups.")

df

Processing group 1 for fifth order assignment
Assigning team 488:f603466f-a7c4-4489-9955-c17a633d5832 (1 pp) to group 1, current count: 175, quota left: 3
Assigning team 236:6d74378d-313d-44fe-a449-4e328a0ffa9b (3 pp) to group 1, current count: 178, quota left: 0
Group 1 has reached quota, stop assigning
Group 3 is already filled to the quota, skipping
Group 4 is already filled to the quota, skipping
Processing group 5 for fifth order assignment
Assigning team 381:1a77bd1a-5708-471f-8e40-37fa76eb6a74 (2 pp) to group 5, current count: 138, quota left: 40
Assigning team 496:e7938115-5d91-40c5-b2ea-a2b43c8f5a1b (2 pp) to group 5, current count: 140, quota left: 38
Assigning team 351:a6b72b6a-c11a-4591-adc3-9ce93863fab2 (1 pp) to group 5, current count: 141, quota left: 37
Assigning team 10:f9d30c30-90c7-429c-9e78-b50ac1bfad65 (1 pp) to group 5, current count: 142, quota left: 36
Assigning team 564:d3386941-1469-4cb3-80fe-841e570dc9c1 (1 pp) to group 5, current count: 143, quota left: 35
A

,id,group_pref,member_count,group_order_1,group_order_2,group_order_3,group_order_4,group_order_5,group_order_6,result
0,ec3766c1-b3ee-4873-9048-b7f7fa17ceba,"6,1,3,5,7,4",2,6,1,3,5,7,4,6
1,738b973c-4cbf-4125-b50e-bcc98f89f77a,"6,1,3,7,4,5",2,6,1,3,7,4,5,6
2,a394c8ef-98d7-49b5-beb3-652b08cd6482,"4,3,6,7,1,5",1,4,3,6,7,1,5,4
3,3620b751-1dcc-4305-8871-b897947ba300,"3,4,7,1,6,5",1,3,4,7,1,6,5,3
4,5602bd74-811a-4e99-9708-179254d9f090,"3,6,7,1,4,5",2,3,6,7,1,4,5,3
...,...,...,...,...,...,...,...,...,...,...
648,959f881a-310f-4e23-b715-1cef16e97869,"3,4,1,5,7,6",1,3,4,1,5,7,6,3
649,b7edde69-726b-4e09-a6c3-9d9b95fd9b4a,"6,5,7,1,3,4",1,6,5,7,1,3,4,5
650,77b3a72b-fd79-4d5b-9537-2ad7df143cbd,"4,6,3,1,7,5",1,4,6,3,1,7,5,4
651,1abaf1d3-b2df-46eb-8ed2-652e98151862,"4,1,6,3,7,5",1,4,1,6,3,7,5,4


# รอบ 6

In [25]:
for group in GROUPS:
    print("="*80)
    # check if group is already filled to the quota already
    if df[df["result"] == group]["member_count"].sum() > GROUP_QUOTA:
        print(f"Group {group} is already filled to the quota, skipping")
        continue
    print(f"Processing group {group} for sixth order assignment")
    # get teams that have group_order_6 as group and not assigned to any group yet
    df_teams_with_group_sixth_order = df[(
        df["group_order_6"] == group) & (df["result"].isnull())]
    # shuffle row order
    df_teams_with_group_sixth_order = df_teams_with_group_sixth_order.sample(
        frac=1)
    current_students_count = df[df["result"] == group]["member_count"].sum()
    for index, row in df_teams_with_group_sixth_order.iterrows():
        if current_students_count < GROUP_QUOTA:
            df.at[index, "result"] = group
            current_students_count += row["member_count"]
            print(
                f"Assigning team {index}:{row['id']} ({row['member_count']} pp) to group {group}, current count: {current_students_count}, quota left: {GROUP_QUOTA - current_students_count}")
        else:
            print(f"Group {group} has reached quota, stop assigning")
            break

unassigned_teams = df[df["result"].isnull()]
if not unassigned_teams.empty:
    print(f"There are {unassigned_teams.shape[0]} teams left unassigned.")
else:
    print("All teams have been assigned to groups.")

unassigned_teams

Processing group 1 for sixth order assignment
Group 3 is already filled to the quota, skipping
Group 4 is already filled to the quota, skipping
Processing group 5 for sixth order assignment
Assigning team 569:cce51d26-87b6-47db-981c-fdd6b68bfe9b (1 pp) to group 5, current count: 150, quota left: 28
Assigning team 267:a49c5fa2-43bc-41cb-9ffc-ce1c1dda9b85 (1 pp) to group 5, current count: 151, quota left: 27
Assigning team 31:6fc714cd-796d-4134-a00a-6383ff740829 (2 pp) to group 5, current count: 153, quota left: 25
Assigning team 67:479d27e9-6be1-4bd3-a55f-bdd83fb7ca1b (3 pp) to group 5, current count: 156, quota left: 22
Assigning team 537:fb690d4b-6c80-434a-9a0e-756640cf180f (3 pp) to group 5, current count: 159, quota left: 19
Assigning team 522:e0362d68-9b41-4dad-8d8d-f993facbd13d (1 pp) to group 5, current count: 160, quota left: 18
Assigning team 60:ca581fcc-6f3a-4ef4-83df-db8637c558da (3 pp) to group 5, current count: 163, quota left: 15
Assigning team 46:614c2c99-db75-4de1-aabd-3

,id,group_pref,member_count,group_order_1,group_order_2,group_order_3,group_order_4,group_order_5,group_order_6,result


In [26]:
# สรุปจำนวนแต่ละ group
for group in GROUPS:
    print(f"Group {group} has {df[df['result'] == group].shape[0]} teams assigned, total students: {df[df['result'] == group]['member_count'].sum()}")

Group 1 has 110 teams assigned, total students: 178
Group 3 has 105 teams assigned, total students: 179
Group 4 has 114 teams assigned, total students: 179
Group 5 has 121 teams assigned, total students: 177
Group 6 has 97 teams assigned, total students: 179
Group 7 has 106 teams assigned, total students: 178


In [27]:
df.to_csv("../data/results_team_processed_with_groups.csv", index=False)